In [1]:
import numpy as np

In [2]:
def random_lattice(n_size=50, init_lattices=1):
    """
    :param n_size: the size of lattice, which you want to create. We interested in
        square lattice: our goal is the chessboard.
    :param init_lattices: how many 2D lattice you want to create.
    :return: the list of 2D lattices.

    lat: lattice
    """
    init_lat_all = []
    for time in range(0, init_lattices):
        init_lat = np.random.randint(2, size=(n_size, n_size))
        init_lat_all.append(init_lat)

    return init_lat_all

In [3]:
n_size = 5
init_lattices = 1

lattice = random_lattice(n_size, init_lattices)[0]

In [4]:
print(lattice)

[[1 1 1 1 1]
 [0 1 0 0 0]
 [1 0 1 0 0]
 [1 0 0 0 1]
 [0 1 1 0 1]]


In [5]:
def create_fitting(D2_lat):
    """
    :param D2_lat:
    :return:
    """
    n_size = len(D2_lat)  # size of lattice of our interest

    
    # --- (1) I punish for having the same state: i-one down, j-one right
    # cells, which we consider
    l_im1_j = np.roll(np.roll(D2_lat, -1, axis=0), 0, axis=1)
    l_i_jm1 = np.roll(np.roll(D2_lat, 0, axis=0), -1, axis=1)
    # punish by: `-3` points
    sel_1a = (D2_lat == l_im1_j).astype(int)
    sel_1b = (D2_lat == l_i_jm1).astype(int)

    
    # sel_1a = np.array(D2_lat + l_ip1_j, dtype=bool)
    # sel_1b = np.array(D2_lat + l_i_jp1, dtype=bool)
    sel_1 = np.array(sel_1a * (-3) + sel_1b * (-3), dtype=int)
    # for which cell we consider `selection 1`: even once
    prep_cells_1a = np.array(sel_1a + sel_1b, dtype=int)
    prep_cells_1b = np.where(prep_cells_1a >= 1, 1, 0)
    prep_cells_1 = np.array(prep_cells_1b, dtype=bool)

    print('sel_1a: one down')
    print(sel_1a)
    print()
    print('sel_1b: one right')
    print(sel_1b)
    print()
    print('sel_1')
    print(sel_1)
    print()
    print('prep_cells_1')
    print(prep_cells_1)
    print()
    print('prep_cells_1a')
    print(prep_cells_1a)
    print()
        
#     return sel_1

    print('------------------------------------------------------------------------------------')

    # --- (2) I give reward for having the same state: (i-one down, j-one left)
    #                                                  (i-one down, j-one right)
    # Also, I punish for having different state on the `diagonal` to the fixed point
    # `1` for fixed cell means we do not consider `selection 1`
    after_1 = (np.zeros((n_size, n_size)) == prep_cells_1).astype(int)
    # cells, which we consider
    l_ip1_jm1 = np.roll(np.roll(D2_lat, -1, axis=0), +1, axis=1)
    l_ip1_jp1 = np.roll(np.roll(D2_lat, -1, axis=0), -1, axis=1)
    # do selection
    sel_2a = (D2_lat == l_ip1_jm1).astype(int)
    sel_2b = (D2_lat == l_ip1_jp1).astype(int)
    # reward by `+8` points
    
    
    sel_2a_p = (np.logical_and(after_1 == sel_2a, after_1 == 1)).astype(int)
    sel_2b_p = (np.logical_and(after_1 == sel_2b, after_1 == 1)).astype(int)
    # punish by `-5 points`
    sel_2a_n = (np.logical_and(after_1 != sel_2a, after_1 == 1)).astype(int)
    sel_2b_n = (np.logical_and(after_1 != sel_2b, after_1 == 1)).astype(int)
    # calculate points
    sel_2 = np.array(sel_2a_p * 8 + sel_2b_p * 8 + sel_2a_n * (-5) + sel_2b_n * (-5), dtype=int)
    
    print('after_1')
    print(after_1)
    print()
    print('sel_2a_p')
    print(sel_2a_p)
    print()
    print('sel_2b_p')
    print(sel_2b_p)
    
    print('sel_2a_n')
    print(sel_2a_n)
    print()
    print('sel_2b_n')
    print(sel_2b_n)
    
    
    

    return sel_1 + sel_2

In [6]:
print(lattice)

[[1 1 1 1 1]
 [0 1 0 0 0]
 [1 0 1 0 0]
 [1 0 0 0 1]
 [0 1 1 0 1]]


In [7]:
# l_im1_j = np.roll(np.roll(lattice, -1, axis=0), 0, axis=1)
# print('sel_1a: one down')
# print(l_im1_j)

In [8]:
# l_i_jp1 = np.roll(np.roll(lattice, 0, axis=0), 1, axis=1)
# print('sel_1b: one right')
# print(l_i_jp1)

In [9]:
fitting_lattice = create_fitting(lattice)

sel_1a: one down
[[0 1 0 0 0]
 [0 0 0 1 1]
 [1 1 0 1 0]
 [0 0 0 1 1]
 [0 1 1 0 1]]

sel_1b: one right
[[1 1 1 1 1]
 [0 0 1 1 1]
 [0 0 0 1 0]
 [0 1 1 0 1]
 [0 1 0 0 0]]

sel_1
[[-3 -6 -3 -3 -3]
 [ 0  0 -3 -6 -6]
 [-3 -3  0 -6  0]
 [ 0 -3 -3 -3 -6]
 [ 0 -6 -3  0 -3]]

prep_cells_1
[[ True  True  True  True  True]
 [False False  True  True  True]
 [ True  True False  True False]
 [False  True  True  True  True]
 [False  True  True False  True]]

prep_cells_1a
[[1 2 1 1 1]
 [0 0 1 2 2]
 [1 1 0 2 0]
 [0 1 1 1 2]
 [0 2 1 0 1]]

------------------------------------------------------------------------------------
after_1
[[0 0 0 0 0]
 [1 1 0 0 0]
 [0 0 1 0 1]
 [1 0 0 0 0]
 [1 0 0 1 0]]

sel_2a_p
[[0 0 0 0 0]
 [1 1 0 0 0]
 [0 0 0 0 1]
 [1 0 0 0 0]
 [0 0 0 0 0]]

sel_2b_p
[[0 0 0 0 0]
 [1 1 0 0 0]
 [0 0 0 0 0]
 [1 0 0 0 0]
 [0 0 0 0 0]]
sel_2a_n
[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 1 0 0]
 [0 0 0 0 0]
 [1 0 0 1 0]]

sel_2b_n
[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 1 0 1]
 [0 0 0 0 0]
 [1 0 0 1 0]]


In [10]:
lattice = [[0,0,0],
           [0,1,0],
           [0,0,0]]

In [11]:
def ngb(lattice):
    # upper row
    l_im1_jm1 = 2**0 * np.roll(np.roll(lattice, -1, axis=0), -1, axis=1)
    l_im1_j = 2**1 * np.roll(np.roll(lattice, -1, axis=0), 0, axis=1)
    l_im1_jp1 = 2**2 * np.roll(np.roll(lattice, -1, axis=0), +1, axis=1)
    # same row
    l_i_jm1 = 2**3 * np.roll(np.roll(lattice, 0, axis=0), -1, axis=1)
    l_i_j = 2**4 * np.roll(np.roll(lattice, 0, axis=0), 0, axis=1)
    l_i_jp1 = 2**5 * np.roll(np.roll(lattice, 0, axis=0), +1, axis=1)
    # lower row
    l_ip1_jm1 = 2**6 * np.roll(np.roll(lattice, +1, axis=0), -1, axis=1)
    l_ip1_j = 2**7 * np.roll(np.roll(lattice, +1, axis=0), 0, axis=1)
    l_ip1_jp1 = 2**8 * np.roll(np.roll(lattice, +1, axis=0), +1, axis=1)

    # Neighbourhood state
    NbgState = l_im1_jm1 + l_im1_j + l_im1_jp1 + l_i_jm1 + l_i_j + l_i_jp1 + l_ip1_jm1 + l_ip1_j + l_ip1_jp1
    return NbgState

In [12]:
ngb_list = ngb(lattice)
print(ngb_list)

[[  1   2   4]
 [  8  16  32]
 [ 64 128 256]]
